# ABEL HiPACE++ example

By Carl A. Lindstrøm (University of Oslo), 8 Sep 2023

### Import ABEL framework

In [1]:
from abel import *
%load_ext autoreload
%autoreload 2

### Define the input beams

In [2]:
# define drive bunch
driver = SourceBasic()
driver.charge = -2e-9 # [C]
driver.energy = 10e9 # [eV]
driver.rel_energy_spread = 0.01
driver.bunch_length = 20e-6 # [m]
driver.z_offset = 100e-6 # [m]
driver.emit_nx, driver.emit_ny = 10e-6, 10e-6 # [m rad]
driver.beta_x, driver.beta_y = 50e-3, 50e-3 # [m]
driver.num_particles = 100000
driver.symmetrize = True

# define stage
stage = StageHipace()
stage.driver_source = driver
stage.length = 0.05 # [m]
stage.plasma_density = 1e22 # [m^-3]
stage.ion_motion = False
stage.beam_ionization = False

In [6]:
# define witness bunch
source = SourceBasic()
source.charge = -0.6e-9 # [C]
source.energy = 100e6 # [eV]
source.rel_energy_spread = 0.01
source.bunch_length = 10e-6 # [m]
source.z_offset = -160e-6 # [m]
source.emit_nx, source.emit_ny = 1e-6, 1e-6 # [m rad]
source.beta_x, source.beta_y = 10e-3, 10e-3 # [m]
source.num_particles = 100000
source.x_offset, source.y_offset = 5e-6, 1e-6 # [m]
source.jitter.x = 5e-6

# define RF accelerator
rf_accelerator = RFAcceleratorBasic()
rf_accelerator.nom_energy_gain = driver.energy-source.energy
rf_accelerator.nom_accel_gradient = 10e6

# define beam delivery system
bds = BeamDeliverySystemBasic()
bds.beta_x = stage.matched_beta_function(source.energy) # [m]
bds.beta_y = stage.matched_beta_function(source.energy) # [m]

# define linac
linac = ConventionalLinac()
linac.source = source
linac.rf_accelerator = rf_accelerator
linac.bds = bds

### Define the experimental setup

In [7]:
# define experiment
experiment = ExperimentPWFA(linac, stage)

### Perform linac tracking

In [8]:
# perform experiment tracking
experiment.stage.num_nodes = 1
beam = experiment.run('hipace_example', overwrite=True)

Tracked #0  SourceBasic             (s =    0.0 m) :   E =   0.1 GeV, Q = -0.60 nC, σz =  10.0 µm, σE =  1.0%, ε =   1.0/1.0 mm-mrad
    ... #1  RFAcceleratorBasic      (s =  990.0 m) :   E =  10.0 GeV, Q = -0.60 nC, σz =  10.0 µm, σE =  0.0%, ε =   1.0/1.0 mm-mrad
    ... #2  BeamDeliverySystemBasic (s = 1308.2 m) :   E =  10.0 GeV, Q = -0.60 nC, σz =  10.0 µm, σE =  0.0%, ε =   1.0/1.0 mm-mrad
>> Finished HiPACE++ (job 8165158): 100%|████████████████████████████████████| 135/135 [00:24<00:00,  5.44 steps/s]


[AbstractIOHandlerImpl] IO Task OPEN_FILE failed with exception. Clearing IO queue and passing on the exception.
[AbstractIOHandlerImpl] IO Task OPEN_FILE failed with exception. Clearing IO queue and passing on the exception.


Error: Read Error in backend HDF5
Object type:	File
Error type:	Inaccessible
Further description:	Supplied directory is not valid: run_data/temp/ca096a72-d798-41c8-8111-cc46b9935ffc/diags/hdf5/

### Plot the wakefield

In [ ]:
# plot wakefield
experiment.stage.plot_wakefield()
experiment.stage.plot_wake()
experiment.stage.plot_evolution()

### Plot the spectrometer screen

### Plot longitudinal phase space

In [ ]:
# plot the longitudinal phase space (at the beginning and end of the stage)
experiment.initial_beam.plot_lps()
experiment.get_beam(-2).plot_lps()